In [ ]:
import pickle

In [ ]:
pickle_files = ["/gpfs/bbp.cscs.ch/data/scratch/proj16/magkanar/nmodl_llvm_benchmark/nmodl/test/benchmark/hh_complete_python_100mil/benchmark_results.pickle"]

In [ ]:
results = {}
for pickle_file in pickle_files:
    with open(pickle_file, 'rb') as handle:
        results.update(pickle.load(handle))
print(results)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
output_dir = "graphs_output"
for modname in results:
    bar_data_state_cpu = {}
    bar_data_cur_cpu = {}
    bar_data_state_gpu = {}
    bar_data_cur_gpu = {}
    for architecture in results[modname]:
        for compiler in results[modname][architecture]:
            if compiler == "nmodl_jit" or compiler == "nmodl_jit_cuda":
                for math_lib_fast_math_flag in results[modname][architecture][compiler]:
                    dict_label = "{}_{}_{}".format(
                            architecture, compiler, math_lib_fast_math_flag
                        )
                    if architecture != "nvptx64":
                        bar_data_state_cpu[dict_label] = results[modname][architecture][compiler][math_lib_fast_math_flag]["nrn_state_hh"][0]
                        bar_data_cur_cpu[dict_label] = results[modname][architecture][compiler][math_lib_fast_math_flag]["nrn_cur_hh"][0]
                    else:
                        bar_data_state_gpu[dict_label] = results[modname][architecture][compiler][math_lib_fast_math_flag]["nrn_state_hh"][0]
                        bar_data_cur_gpu[dict_label] = results[modname][architecture][compiler][math_lib_fast_math_flag]["nrn_cur_hh"][0]
            else:
                for flags in results[modname][architecture][compiler]:
                    dict_label = "{}_{}_{}".format(architecture, compiler, flags)
                    if architecture != "nvptx64":
                        bar_data_state_cpu[dict_label] = results[modname][architecture][compiler][flags]["nrn_state_ext"][0]
                        bar_data_cur_cpu[dict_label] = results[modname][architecture][compiler][flags]["nrn_cur_ext"][0]
                    else:
                        bar_data_state_gpu[dict_label] = results[modname][architecture][compiler][flags]["nrn_state_ext"][0]
                        bar_data_cur_gpu[dict_label] = results[modname][architecture][compiler][flags]["nrn_cur_ext"][0]
    state_keys_cpu = list(bar_data_state_cpu.keys())
    state_vals_cpu = [float(bar_data_state_cpu[k]) for k in state_keys_cpu]
    cur_keys_cpu = list(bar_data_cur_cpu.keys())
    cur_vals_cpu = [float(bar_data_cur_cpu[k]) for k in cur_keys_cpu]
    state_keys_gpu = list(bar_data_state_gpu.keys())
    state_vals_gpu = [float(bar_data_state_gpu[k]) for k in state_keys_gpu]
    cur_keys_gpu = list(bar_data_cur_gpu.keys())
    cur_vals_gpu = [float(bar_data_cur_gpu[k]) for k in cur_keys_gpu]
    # if state_vals is not empty
    if len(state_vals_cpu) > 0:
        state_barplot = sns.barplot(x=state_vals_cpu, y=state_keys_cpu, orient="h")
        state_barplot.tick_params(labelsize=6)
        plt.xlabel("Runtime (s)")
        plt.ylabel("Compiler setup")
        plt.title("nrn_state runtime for {} on CPU".format(modname))
        plt.savefig("{}/{}_state_benchmark_cpu.pdf".format(output_dir, modname), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()
    else:
        print("No results for state kernel of {} for CPU".format(modname))
    if len(cur_vals_cpu) > 0:
        cur_barplot = sns.barplot(x=cur_vals_cpu, y=cur_keys_cpu, orient="h", label="nrn_cur runtime for {}".format(modname))
        cur_barplot.tick_params(labelsize=6)
        plt.xlabel("Runtime (s)")
        plt.ylabel("Compiler setup")
        plt.title("nrn_cur runtime for {} on CPU".format(modname))
        plt.savefig("{}/{}_cur_benchmark_cpu.pdf".format(output_dir, modname), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()
    else:
        print("No results for current kernel of {} for CPU".format(modname))
    # if state_vals is not empty
    if len(state_vals_gpu) > 0:
        state_barplot = sns.barplot(x=state_vals_gpu, y=state_keys_gpu, orient="h", label="nrn_state runtime for {}".format(modname))
        state_barplot.tick_params(labelsize=6)
        plt.xlabel("Runtime (s)")
        plt.ylabel("Compiler setup")
        plt.title("nrn_state runtime for {} on GPU".format(modname))
        plt.savefig("{}/{}_state_benchmark_gpu.pdf".format(output_dir, modname), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()
    else:
        print("No results for state kernel of {} for GPU".format(modname))
    if len(cur_vals_gpu) > 0:
        cur_barplot = sns.barplot(x=cur_vals_gpu, y=cur_keys_gpu, orient="h", label="nrn_cur runtime for {}".format(modname))
        cur_barplot.tick_params(labelsize=6)
        plt.xlabel("Runtime (s)")
        plt.ylabel("Compiler setup")
        plt.title("nrn_cur runtime for {} on GPU".format(modname))
        plt.savefig("{}/{}_cur_benchmark_gpu.pdf".format(output_dir, modname), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()
    else:
        print("No results for current kernel of {} for GPU".format(modname))


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import json

output_dir = "graphs_output"
compiler_flags = None

def _get_flags_string(flags):
    return flags.replace(" ", "_").replace('-','').replace('=','_')

def generate_graph(compilers_comparison_config, graph_suffix):
    compiler_flags = json.loads(compilers_comparison_config)
    for modname in results:
        bar_data_state_cpu = {}
        bar_data_cur_cpu = {}
        for architecture in results[modname]:
            for compiler in results[modname][architecture]:
                if compiler in compiler_flags and architecture in compiler_flags[compiler]:
                    for flags in compiler_flags[compiler][architecture]:
                        dict_label = "{}_{}_{}".format(architecture, compiler, _get_flags_string(flags))
                        if compiler == "nmodl_jit":
                            state_kernel_name = "nrn_state_{}".format(modname.replace("-", "_"))
                            cur_kernel_name = "nrn_cur_{}".format(modname.replace("-", "_"))
                        else:
                            state_kernel_name = "nrn_state_ext"
                            cur_kernel_name = "nrn_cur_ext"
                        bar_data_state_cpu[dict_label] = results[modname][architecture][compiler][_get_flags_string(flags)][state_kernel_name][0]
                        bar_data_cur_cpu[dict_label] = results[modname][architecture][compiler][_get_flags_string(flags)][state_kernel_name][0]
        state_keys_cpu = list(bar_data_state_cpu.keys())
        state_vals_cpu = [float(bar_data_state_cpu[k]) for k in state_keys_cpu]
        cur_keys_cpu = list(bar_data_cur_cpu.keys())
        cur_vals_cpu = [float(bar_data_cur_cpu[k]) for k in cur_keys_cpu]
        # if state_vals is not empty
        if len(state_vals_cpu) > 0:
            state_barplot = sns.barplot(x=state_vals_cpu, y=state_keys_cpu, orient="h")
            state_barplot.tick_params(labelsize=6)
            plt.xlabel("Runtime (s)")
            plt.ylabel("Compiler setup")
            plt.title("nrn_state runtime for {} on CPU".format(modname))
            plt.savefig("{}/{}_state_benchmark_cpu_{}.pdf".format(output_dir, modname, graph_suffix), format="pdf", bbox_inches="tight")
            plt.show()
            plt.close()
        else:
            print("No results for state kernel of {} for CPU".format(modname))
        if len(cur_vals_cpu) > 0:
            cur_barplot = sns.barplot(x=cur_vals_cpu, y=cur_keys_cpu, orient="h", label="nrn_cur runtime for {}".format(modname))
            cur_barplot.tick_params(labelsize=6)
            plt.xlabel("Runtime (s)")
            plt.ylabel("Compiler setup")
            plt.title("nrn_cur runtime for {} on CPU".format(modname))
            plt.savefig("{}/{}_cur_benchmark_cpu_{}.pdf".format(output_dir, modname, graph_suffix), format="pdf", bbox_inches="tight")
            plt.show()
            plt.close()
        else:
            print("No results for current kernel of {} for CPU".format(modname))


In [ ]:
compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div"
    ],
    "nehalem": [
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "clang": {
    "default": [
      "-O3 -ffast-math",
      "-O3 -ffast-math jit"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp -fveclib=SVML jit"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML jit"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML jit"
    ]
  },
  "gcc": {
    "default": [
      "-O3 -ffast-math -ftree-vectorize"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn"
    ],
    "nehalem": [
      "SVML_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn"
    ]
  }
}
"""
generate_graph(compilers_comparison_config, "all_compilers")

In [ ]:
compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div"
    ],
    "nehalem": [
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn"
    ],
    "nehalem": [
      "SVML_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn"
    ]
  }
}
"""
generate_graph(compilers_comparison_config, "intel_vs_nmodl")

In [ ]:
import pandas as pd

def generate_graph_pandas(compilers_comparison_config, graph_suffix, output_dir):
    df = None
    bar_data_state_cpu_panda = {}
    bar_data_state_cpu_panda["architecture"] = []
    bar_data_state_cpu_panda["compiler"] = []
    bar_data_state_cpu_panda["runtime"] = []
    bar_data_cur_cpu_panda = {}
    bar_data_cur_cpu_panda["architecture"] = []
    bar_data_cur_cpu_panda["compiler"] = []
    bar_data_cur_cpu_panda["runtime"] = []
    compiler_flags = json.loads(compilers_comparison_config)
    for modname in results:
        for architecture in results[modname]:
            bar_data_state_cpu_panda[architecture] = {}
            for compiler in results[modname][architecture]:
                if compiler in compiler_flags and architecture in compiler_flags[compiler]:
                    for flags in compiler_flags[compiler][architecture]:
                        dict_label = "{}_{}_{}".format(architecture, compiler, _get_flags_string(flags))
                        if compiler == "nmodl_jit":
                            state_kernel_name = "nrn_state_{}".format(modname.replace("-", "_"))
                            cur_kernel_name = "nrn_cur_{}".format(modname.replace("-", "_"))
                        else:
                            state_kernel_name = "nrn_state_ext"
                            cur_kernel_name = "nrn_cur_ext"
                        if compiler == "clang" and "jit" in flags:
                            bar_data_state_cpu_panda["architecture"].append(architecture)
                            bar_data_state_cpu_panda["compiler"].append("clang_jit")
                            bar_data_state_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][state_kernel_name][0])
                            bar_data_cur_cpu_panda["architecture"].append(architecture)
                            bar_data_cur_cpu_panda["compiler"].append("clang_jit")
                            bar_data_cur_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][cur_kernel_name][0])
                        else:
                            bar_data_state_cpu_panda["architecture"].append(architecture)
                            bar_data_state_cpu_panda["compiler"].append(compiler)
                            bar_data_state_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][state_kernel_name][0])
                            bar_data_cur_cpu_panda["architecture"].append(architecture)
                            bar_data_cur_cpu_panda["compiler"].append(compiler)
                            bar_data_cur_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][cur_kernel_name][0])
        df_state = pd.DataFrame(bar_data_state_cpu_panda, columns=["architecture", "compiler", "runtime"])
        sns.barplot(x='architecture', y='runtime', hue='compiler', data=df_state)
        plt.xlabel("Architecture")
        plt.ylabel("Runtime (s)")
        plt.title("nrn_state runtime for {}".format(modname))
        plt.savefig("{}/{}_state_benchmark_{}.pdf".format(output_dir, modname, graph_suffix), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()
        df_cur = pd.DataFrame(bar_data_cur_cpu_panda, columns=["architecture", "compiler", "runtime"])
        sns.barplot(x='architecture', y='runtime', hue='compiler', data=df_cur)
        plt.xlabel("Architecture")
        plt.ylabel("Runtime (s)")
        plt.title("nrn_cur runtime for {}".format(modname))
        plt.savefig("{}/{}_cur_benchmark_{}.pdf".format(output_dir, modname, graph_suffix), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()

compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div"
    ],
    "nehalem": [
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "clang": {
    "default": [
      "-O3 -ffast-math",
      "-O3 -ffast-math jit"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp -fveclib=SVML jit"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML jit"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML jit"
    ]
  },
  "gcc": {
    "default": [
      "-O3 -ffast-math -ftree-vectorize"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn"
    ],
    "nehalem": [
      "SVML_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn"
    ]
  }
}
"""

generate_graph_pandas(compilers_comparison_config, "cpu_all_compilers", "graphs_output_pandas")

In [ ]:
compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div"
    ],
    "nehalem": [
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn"
    ],
    "nehalem": [
      "SVML_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn"
    ]
  }
}
"""
generate_graph_pandas(compilers_comparison_config, "cpu_intel_vs_nmodl", "graphs_output_pandas")

In [ ]:
compilers_comparison_config = """
{
  "nvhpc": {
    "nvptx64": [
      "-O3 -gpu=nordc,fastmath"
    ]
  },
  "nmodl_jit": {
    "nvptx64": [
      "libdevice_nnancontractafn"
    ]
  }
}
"""
generate_graph_pandas(compilers_comparison_config, "gpu_openacc_vs_nmodl", "graphs_output_pandas")